In [ ]:
%matplotlib widget

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm

import lotr.plotting as pltltr
from lotr import LotrExperiment, dataset_folders
from lotr.data_preprocessing.anatomy import anatomical_angle_remapping


from circle_fit import hyper_fit
from sklearn.decomposition import PCA

COLS = pltltr.COLS

In [ ]:
results_list = []

# Loop over all fish, and compute values for data and after shuffling coords itentity:
for path in dataset_folders:
    exp = LotrExperiment(path)

    # center coords (will be replaced with morphing):
    coords = exp.morphed_coords_um[:, :]
    coords[:, 1] = coords[:, 1] + 16  # arbitrary centering

    sel = np.full(exp.n_rois, False)

    real_angles = np.full(exp.n_rois, np.nan)
    real_angles[exp.hdn_indexes] = exp.rpc_angles
    sel[exp.hdn_indexes] = True

    # And, for plotting, computing an "anatomical phase", aka position around the anatomy:
    anatomical_phase = np.angle(-coords[:, 1] + 1j * -coords[:, 0])

    # Stretch angle to avoid undersampling aroung midline
    anatomical_phase = anatomical_angle_remapping(anatomical_phase)

    r = np.mean(np.sqrt(exp.rpc_scores[:, 0] ** 2 + exp.rpc_scores[:, 1] ** 2))
    results_list.append(
        pd.DataFrame(
            dict(
                dv_pos=coords[:, 0],
                lr_pos=coords[:, 1],
                ap_pos=coords[:, 2],
                real_angles=real_angles,
                exp_id=path.name,
                is_ring=sel,
            )
        )
    )

pooled_df = pd.concat(results_list, axis=0).reset_index()

In [ ]:
pooled_df["ot"] = (
    np.sqrt((pooled_df["lr_pos"] - 15) ** 2 + (pooled_df["ap_pos"] + 165) ** 2) > 180
)
pooled_df["ahb"] = (
    np.sqrt((pooled_df["lr_pos"] - 15) ** 2 + (pooled_df["ap_pos"] + 165) ** 2) < 180
) & (np.sqrt((pooled_df["lr_pos"] - 15) ** 2 + (pooled_df["ap_pos"] + 250) ** 2) > 194)

pooled_df["rh2"] = (
    np.sqrt((pooled_df["lr_pos"] - 15) ** 2 + (pooled_df["ap_pos"] + 250) ** 2) < 194
)

In [ ]:
analyzed_dict = dict()
groups = ["ahb", "rh2", "ot"]
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)

    exp_dict = dict()
    for group in groups:
        # Compute PCA and transform traces:
        sel = pooled_df[pooled_df["exp_id"] == path.name]
        traces = exp.traces[exp.pca_t_slice, sel[group]].T
        pca = PCA(n_components=5).fit(traces)
        pcaed = pca.transform(traces)

        # Fit circle:
        circle_params = hyper_fit(pcaed[:, :2])

        # Compute phase, after subtracting center of the circle
        angles = np.arctan2(
            pcaed[:, 1] - circle_params[1], pcaed[:, 0] - circle_params[0]
        )

        exp_dict[group] = dict(
            pcaed=pcaed,
            min_corrs=np.corrcoef(exp.traces[exp.pca_t_slice, sel[group]].T).min(0),
        )

    analyzed_dict[path.name] = exp_dict

In [ ]:
plt.close("all")

titles = dict(ahb="aHB", ot="Optic tectum", rh2="Rhomb. 2")

f, all_axs = plt.subplots(
    11,
    12,
    figsize=(9, 12),
    gridspec_kw=dict(
        top=0.88,
        right=1,
        bottom=0.01,
        hspace=0.3,
        wspace=0,
        width_ratios=[1, 1, 1, 0.3] * 3,
    ),
    sharey=True,
    sharex=True,
)

axs_flat = all_axs.flatten()
[ax.axis("off") for ax in axs_flat]


for i, group in enumerate(groups):
    ax = f.add_axes((i * 0.15 + 0.4, 0.91, 0.07, 0.07))

    ax.scatter(
        pooled_df["lr_pos"],
        pooled_df["ap_pos"],
        s=1,
        alpha=0.1,
        color=".6",
        lw=0,
        rasterized=True,
    )
    ax.scatter(
        pooled_df.loc[pooled_df[group], "lr_pos"],
        pooled_df.loc[pooled_df[group], "ap_pos"],
        s=1,
        alpha=0.05,
        color="C0",
        lw=0,
        rasterized=True,
    )
    pltltr.despine(ax, "all")
    ax.set_title(titles[group])

for i in range(len(dataset_folders)):
    path = dataset_folders[i]
    i_plot = i * 4
    axs = axs_flat[i_plot : i_plot + 4]
    for group, ax in zip(groups, axs):
        pcaed = analyzed_dict[path.name][group]["pcaed"]
        min_c = analyzed_dict[path.name][group]["min_corrs"]
        sc = ax.scatter(
            pcaed[:, 0],
            pcaed[:, 1],
            c=min_c,
            vmin=-0.8,
            vmax=0.8,
            cmap="coolwarm",
            s=1,
            rasterized=True,
        )

        if i < 3:
            ax.set_title(titles[group], fontsize=8)
    axs[1].text(0, 70, LotrExperiment(path).exp_code, ha="center")

    pltltr.add_scalebar(
        axs[1], xlen=40, ylen=40, xpos=-70, ypos=-70, xlabel="PC1", ylabel="PC2"
    )
pltltr.add_cbar(
    sc,
    axs_flat[i * 4 + 5],
    inset_loc=(0, 0.5, 1, 0.13),
    orientation="horizontal",
    title="Min. corr.",
    titlesize=8,
    ticks=[-0.5, 0, 0.5],
)

In [ ]:
pltltr.savefig("anatomical_roi_selection")

In [ ]:
analyzed_dict = dict()
groups = ["ahb", "rh2", "ot"]
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)

    exp_dict = dict()
    for group in groups:
        # Compute PCA and transform traces:
        sel = pooled_df[pooled_df["exp_id"] == path.name]
        traces = exp.traces[exp.pca_t_slice, sel[group]].T
        cc = np.corrcoef(traces)
        sel_traces = traces[np.min(cc, 0) < -0.4, :]

        if sel_traces.shape[0] > 5:
            pca = PCA(n_components=5).fit(sel_traces)
            pcaed = pca.transform(traces)
        else:
            pcaed = np.full(traces.shape, np.nan)

        # Fit circle:
        circle_params = hyper_fit(pcaed[:, :2])

        # Compute phase, after subtracting center of the circle
        angles = np.arctan2(
            pcaed[:, 1] - circle_params[1], pcaed[:, 0] - circle_params[0]
        )

        exp_dict[group] = dict(
            pcaed=pcaed,
            min_corrs=np.corrcoef(exp.traces[exp.pca_t_slice, sel[group]].T).min(0),
        )

    analyzed_dict[path.name] = exp_dict

In [ ]:
sel_traces.shape

In [ ]:
cc.min()

In [ ]:
plt.close("all")

titles = dict(ahb="aHB", ot="Optic tectum", rh2="Rhomb. 2")

f, all_axs = plt.subplots(
    11,
    12,
    figsize=(9, 12),
    gridspec_kw=dict(
        top=0.88,
        right=1,
        bottom=0.01,
        hspace=0.3,
        wspace=0,
        width_ratios=[1, 1, 1, 0.3] * 3,
    ),
    sharey=True,
    sharex=True,
)

axs_flat = all_axs.flatten()
[ax.axis("off") for ax in axs_flat]


for i, group in enumerate(groups):
    ax = f.add_axes((i * 0.15 + 0.4, 0.91, 0.07, 0.07))

    ax.scatter(
        pooled_df["lr_pos"],
        pooled_df["ap_pos"],
        s=1,
        alpha=0.1,
        color=".6",
        lw=0,
        rasterized=True,
    )
    ax.scatter(
        pooled_df.loc[pooled_df[group], "lr_pos"],
        pooled_df.loc[pooled_df[group], "ap_pos"],
        s=1,
        alpha=0.05,
        color="C0",
        lw=0,
        rasterized=True,
    )
    pltltr.despine(ax, "all")
    ax.set_title(titles[group])

for i in range(len(dataset_folders)):
    path = dataset_folders[i]
    i_plot = i * 4
    axs = axs_flat[i_plot : i_plot + 4]
    for group, ax in zip(groups, axs):
        pcaed = analyzed_dict[path.name][group]["pcaed"]
        min_c = analyzed_dict[path.name][group]["min_corrs"]
        sc = ax.scatter(
            pcaed[:, 0],
            pcaed[:, 1],
            c=min_c,
            vmin=-0.8,
            vmax=0.8,
            cmap="coolwarm",
            s=1,
            rasterized=True,
        )

        if i < 3:
            ax.set_title(titles[group], fontsize=8)
    axs[1].text(0, 70, LotrExperiment(path).exp_code, ha="center")

    pltltr.add_scalebar(
        axs[1], xlen=40, ylen=40, xpos=-70, ypos=-70, xlabel="PC1", ylabel="PC2"
    )
pltltr.add_cbar(
    sc,
    axs_flat[i * 4 + 5],
    inset_loc=(0, 0.5, 1, 0.13),
    orientation="horizontal",
    title="Min. corr.",
    titlesize=8,
    ticks=[-0.5, 0, 0.5],
)